<a href="https://colab.research.google.com/github/dkomlen/finance-ml/blob/main/yield_curve_construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import quandl

#Import Model Packages 
from sklearn.decomposition import PCA

#### Loading the data

In [4]:
quandl.ApiConfig.api_key  =  'XXX'

In [6]:
treasury = ['FRED/DGS1MO',
           'FRED/DGS3MO',
           'FRED/DGS6MO',
           'FRED/DGS1',
           'FRED/DGS2',
           'FRED/DGS3',
           'FRED/DGS5',
           'FRED/DGS7',
           'FRED/DGS10',
           'FRED/DGS20',
           'FRED/DGS30']

treasury_df = quandl.get(treasury)

In [7]:
treasury_df.columns = ['TRESY1mo',
                    'TRESY3mo', 
                    'TRESY6mo',
                    'TRESY1y',
                    'TRESY2y',
                    'TRESY3y',
                    'TRESY5y',
                    'TRESY7y',
                    'TRESY10y',
                    'TRESY20y',
                    'TRESY30y']

In [8]:
treasury_df

,TRESY1mo,TRESY3mo,TRESY6mo,TRESY1y,TRESY2y,TRESY3y,TRESY5y,TRESY7y,TRESY10y,TRESY20y,TRESY30y
Date,,,,,,,,,,,
1962-01-02,NaN,NaN,NaN,3.22,NaN,3.70,3.88,NaN,4.06,4.07,NaN
1962-01-03,NaN,NaN,NaN,3.24,NaN,3.70,3.87,NaN,4.03,4.07,NaN
1962-01-04,NaN,NaN,NaN,3.24,NaN,3.69,3.86,NaN,3.99,4.06,NaN
1962-01-05,NaN,NaN,NaN,3.26,NaN,3.71,3.89,NaN,4.02,4.07,NaN
1962-01-08,NaN,NaN,NaN,3.31,NaN,3.71,3.91,NaN,4.03,4.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-03-21,0.22,0.54,0.95,1.40,2.14,2.34,2.33,2.36,2.32,2.67,2.55
2022-03-22,0.20,0.51,0.96,1.59,2.18,2.38,2.39,2.42,2.38,2.71,2.60
2022-03-23,0.15,0.50,0.93,1.52,2.13,2.32,2.34,2.37,2.32,2.65,2.52


In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
dataset = treasury_df

#### 3.1. Descriptive Statistics

In [11]:
dataset.shape


(15044, 11)

In [12]:
set_option('display.width', 100)
dataset.tail(5)

,TRESY1mo,TRESY3mo,TRESY6mo,TRESY1y,TRESY2y,TRESY3y,TRESY5y,TRESY7y,TRESY10y,TRESY20y,TRESY30y
Date,,,,,,,,,,,
2022-03-21,0.22,0.54,0.95,1.40,2.14,2.34,2.33,2.36,2.32,2.67,2.55
2022-03-22,0.20,0.51,0.96,1.59,2.18,2.38,2.39,2.42,2.38,2.71,2.60
2022-03-23,0.15,0.50,0.93,1.52,2.13,2.32,2.34,2.37,2.32,2.65,2.52
2022-03-24,0.16,0.52,0.96,1.55,2.13,2.35,2.37,2.39,2.34,2.63,2.51
2022-03-25,0.17,0.55,1.01,1.67,2.30,2.51,2.55,2.56,2.48,2.74,2.60
